In [ ]:

%run includes/includes.ipynb


In [ ]:

# show header

owner = owner.value
repo = repo.value
start_date = start_date.value.strftime('%Y-%m-%d')
end_date = end_date.value.strftime('%Y-%m-%d')
header_text = f"# Sprint {sprint_number.value}\n"
header_text += f"#### {start_date} to {end_date}\n"
display(Markdown(header_text))

In [ ]:
%%stackql
SELECT 
SUBSTRING(sha, 1, 7) as sha, 
JSON_EXTRACT(commit, '$$.author.name') as author,  
DATE(JSON_EXTRACT(commit, '$$.author.date')) as date
FROM github.repos.commits
WHERE owner = '$owner' AND repo = '$repo' AND
date > '$start_date' AND date <= '$end_date'

In [ ]:
%%stackql --no-display
SELECT login, contributions FROM github.repos.contributors WHERE owner = '$owner' AND repo = '$repo'

In [ ]:
show_contributions(stackql_df)

In [ ]:
%%stackql
SELECT 
date(published_at) as published_at,
target_commitish,
tag_name
FROM github.repos.releases 
WHERE 
    owner = '$owner' AND 
    repo = '$repo' AND
    published_at > '$start_date' AND published_at <= '$end_date'

In [ ]:
%%stackql
SELECT 
    number, 
    JSON_EXTRACT(head, '$$.ref') as ref,
    date(created_at) as created_at,
    updated_at
FROM 
    github.pulls.pull_requests
WHERE 
    owner = '$owner' AND 
    repo = '$repo' AND
    created_at > '$start_date' AND created_at <= '$end_date'

In [ ]:
%%stackql
SELECT 
activity_type,
date(timestamp) as timestamp,
JSON_EXTRACT(actor, '$$.login') as login
FROM
github.repos.activities
WHERE 
    owner = '$owner' AND 
    repo = '$repo'  AND
    timestamp > '$start_date' AND timestamp <= '$end_date'

In [ ]:
%%stackql
SELECT 
JSON_EXTRACT(t.value, '$$.type') as type,
JSON_EXTRACT(t.value, '$$.path') as path,
JSON_EXTRACT(t.value, '$$.size') as size,
JSON_EXTRACT(t.value, '$$.sha') as sha
FROM github.git.trees, json_each("tree") t
WHERE tree_sha = '$sha' 
AND owner = '$owner' AND repo = '$repo'